In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import math

from dateutil.relativedelta import *
from datetime import date
from datetime import datetime

def formatarStr (DataFrame, columns):
    for col in columns:
        DataFrame[col] = DataFrame[col].str.strip()
        
def calcularSomaIdade(idadeStr):
    try: 
        df_soma = df_result.query("idade == \""+idadeStr+"\"" )
        total= df_soma['qtde'].sum()    
        return total
    except (RuntimeError, TypeError, NameError, ValueError):
        return np.nan

In [2]:
#ISO-8859-1
nomeArquivoOrigemIdade = 'datasets/origem/MPS/ACT03.csv'
nomeArquivoResultadoIdadeTotal = 'datasets/result/dados_idade_MPS_t.csv'
nomeArquivoResultadoIdade = 'datasets/result/dados_idade_MPS.csv'

columns_types = {'ano': str, 'idade': str, 'motivo"': str, 'sexo': str, 'qtde':str, 'situacao': str}

nomes  = ['ano', 'idade', 'motivo', 'sexo', 'qtde', 'situacao']

dados_originais = pd.read_csv(nomeArquivoOrigemIdade, sep = ',', header=0, names=nomes, dtype=columns_types, 
                              encoding='ISO-8859-1')

df_original = pd.DataFrame(data = dados_originais, columns=nomes)
df_original.shape[0] #10112

10112

In [3]:
df_original.head()

,ano,idade,motivo,sexo,qtde,situacao
0,1997,Até 19 Anos,Típico-Com Cat,Masculino,7581,Com Cat Registrada
1,1997,Até 19 Anos,Típico-Com Cat,Feminino,1061,Com Cat Registrada
2,1997,Até 19 Anos,Típico-Com Cat,Ignorado,18851,Com Cat Registrada
3,1997,20 a 24 Anos,Típico-Com Cat,Masculino,15847,Com Cat Registrada
4,1997,20 a 24 Anos,Típico-Com Cat,Feminino,1934,Com Cat Registrada


In [4]:
df_original.drop(['motivo', 'sexo'], axis=1, inplace = True)
formatarStr(df_original, ['ano', 'idade', 'qtde', 'situacao'])

selecaoRegistrados = df_original['situacao'] == 'Com Cat Registrada'
df_original = df_original[selecaoRegistrados]
df_original.shape[0] #6827

6827

In [13]:
df_original.drop(['situacao'], axis=1, inplace = True)

df_original.dropna(subset = ['qtde', 'ano'], inplace = True)

df_original['ano'] = df_original['ano'].apply(pd.to_numeric)
df_original['qtde'] = df_original['qtde'].apply(pd.to_numeric)

selecao2016 = df_original['ano'] >= 2016
df_original = df_original[selecao2016]

#excluir os registros cujo cbo foi preenchido com 'Ignorado'
selecaoIgnorado = df_original['idade'] != 'Ignorada'
df_original = df_original[selecaoIgnorado]
df_original.shape[0] #1260

KeyError: "['situacao'] not found in axis"

In [14]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ano     1260 non-null   int64 
 1   idade   1260 non-null   object
 2   qtde    1260 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 29.7+ KB


In [15]:
df_original.index = range(df_original.shape[0]) #0:5511
df_original.head(20)

,ano,idade,qtde
0,2016,Até 19 Anos,3
1,2016,Até 19 Anos,4
2,2016,Até 19 Anos,1
3,2016,Até 19 Anos,25
4,2016,Até 19 Anos,9
5,2016,Até 19 Anos,16
6,2016,Até 19 Anos,14
7,2016,Até 19 Anos,138
8,2016,Até 19 Anos,1
9,2016,Até 19 Anos,96


In [16]:
df_result = pd.DataFrame(df_original.groupby(['idade', 'ano'], as_index=False)['qtde'].sum())
df_result.head(40)

,idade,ano,qtde
0,20 a 24 Anos,2016,67227
1,20 a 24 Anos,2017,63412
2,20 a 24 Anos,2018,67976
3,25 a 29 Anos,2016,80415
4,25 a 29 Anos,2017,73725
5,25 a 29 Anos,2018,76255
6,30 a 34 Anos,2016,81946
7,30 a 34 Anos,2017,75967
8,30 a 34 Anos,2018,79589
9,35 a 39 Anos,2016,72787


In [17]:
df_result.shape[0] #36

36

In [18]:
df_result.to_csv(nomeArquivoResultadoIdade, sep = ';', encoding='UTF-8')

In [19]:
df_resultGroup = pd.DataFrame({'idade': df_result['idade']})
df_resultGroup.drop_duplicates(subset ="idade", keep = 'first', inplace = True)
df_resultGroup['total'] = df_resultGroup['idade'].apply(lambda x: calcularSomaIdade(x))
df_resultGroup = df_resultGroup.sort_values(['total'], ascending = False)

df_resultGroup.index = range(df_resultGroup.shape[0]) #0:47
df_resultGroup.head(30)

,idade,total
0,30 a 34 Anos,237502
1,25 a 29 Anos,230395
2,35 a 39 Anos,218599
3,20 a 24 Anos,198615
4,40 a 44 Anos,165482
5,45 a 49 Anos,128847
6,50 a 54 Anos,99241
7,55 a 59 Anos,60189
8,Até 19 Anos,33722
9,60 a 64 Anos,26831


In [20]:
df_resultGroup.to_csv(nomeArquivoResultadoIdadeTotal, sep = ';', encoding='UTF-8')